<a href="https://colab.research.google.com/github/huricane85/fpl2020_python/blob/master/Copy_of_FDR_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import scipy
import numpy as np
from numpy import random
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
epl_table = pd.read_csv('/content/drive/My Drive/FPL/Forecasting/Data/EPL_Table.csv', encoding = "ISO-8859-1", engine='python')
soccer_spi = pd.read_csv('/content/drive/My Drive/FPL/Forecasting/Data/soccer-spi/spi_matches.csv', encoding = "ISO-8859-1", engine='python')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
epl_table

,Season,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts,Promoted,Relegated
0,19_20,1,Liverpool,38,32,3,3,85,33,52,99,NaN,NaN
1,19_20,2,Manchester City,38,26,3,9,102,35,67,81,NaN,NaN
2,19_20,3,Manchester United,38,18,12,8,66,36,30,66,NaN,NaN
3,19_20,4,Chelsea,38,20,6,12,69,54,15,66,NaN,NaN
4,19_20,5,Leicester City,38,18,8,12,67,41,26,62,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,10_11,16,Wigan Athletic,38,9,15,14,40,61,-21,42,NaN,NaN
196,10_11,17,Wolverhampton Wanderers,38,11,7,20,46,66,-20,40,NaN,NaN
197,10_11,18,Birmingham City,38,8,15,15,37,58,-21,39,NaN,R
198,10_11,19,Blackpool,38,10,9,19,55,78,-23,39,P3,R


In [ ]:
win_lose_draw = epl_table.groupby(epl_table["Season"]).sum().drop(columns = ["Pos", "Pld", "GF", "Pts", "GD"]).rename(columns={'GA': 'Goals'})

In [ ]:
win_lose_draw["D"] = win_lose_draw["D"].div(2)

In [ ]:
win_lose_draw

,W,D,L,Goals
Season,,,,
10_11,269,111.0,269,1063
11_12,287,93.0,287,1066
12_13,272,108.0,272,1063
13_14,302,78.0,302,1052
14_15,287,93.0,287,975
15_16,273,107.0,273,1026
16_17,296,84.0,296,1064
17_18,281,99.0,281,1018
18_19,309,71.0,309,1072


In [ ]:
win_lose_draw["percent_result"] = win_lose_draw.apply(lambda row: row.W/380, axis=1)
win_lose_draw["percent_draw"] = win_lose_draw.apply(lambda row: row.D/380, axis=1)

In [ ]:
win_lose_draw

,W,D,L,Goals,percent_result,percent_draw
Season,,,,,,
10_11,269,111.0,269,1063,0.707895,0.292105
11_12,287,93.0,287,1066,0.755263,0.244737
12_13,272,108.0,272,1063,0.715789,0.284211
13_14,302,78.0,302,1052,0.794737,0.205263
14_15,287,93.0,287,975,0.755263,0.244737
15_16,273,107.0,273,1026,0.718421,0.281579
16_17,296,84.0,296,1064,0.778947,0.221053
17_18,281,99.0,281,1018,0.739474,0.260526
18_19,309,71.0,309,1072,0.813158,0.186842


In [ ]:
average_chance_of_draw = win_lose_draw["percent_draw"].mean()
average_chance_of_a_result = 1 - average_chance_of_draw

In [ ]:
average_chance_of_draw

0.24631578947368418

In [ ]:
average_chance_of_a_result

0.7536842105263158

In [ ]:
epl_filter = soccer_spi["league"] == "Barclays Premier League"

In [ ]:
soccer_spi_epl = soccer_spi[epl_filter]

In [ ]:
home_xG = soccer_spi_epl.groupby(["team1", "season"]).mean().reset_index()
home_xG.drop(home_xG.columns.difference(["team1", "season", "score1", "score2", "xg1", "xg2", "nsxg1", "nsxg2"]), 1, inplace=True)
home_xG = home_xG.rename(columns={'team1': 'home_team', 'score1': 'G_home', 'score2': 'Gc_home', 'xg1': 'xG_home', 'xg2': 'xGc_home', 'nsxg1': 'non_shot_xG_home', 'nsxg2': 'non_shot_xGc_home'})

In [ ]:
away_xG = soccer_spi_epl.groupby(["team2", "season"]).mean().reset_index()
away_xG.drop(away_xG.columns.difference(["team2", "season", "score1", "score2", "xg1", "xg2", "nsxg1", "nsxg2"]), 1, inplace=True)
away_xG = away_xG.rename(columns={'team2': 'away_team', 'score2': 'G_away', 'score1': 'Gc_away', 'xg2': 'xG_away', 'xg1': 'xGc_away', 'nsxg1': 'non_shot_xG_away', 'nsxg2': 'non_shot_xGc_away'})

In [ ]:
home_xG["home_index"] = home_xG["home_team"] + " " + home_xG["season"].astype(str)

In [ ]:
away_xG = away_xG[['away_team', 'season', 'G_away', 'Gc_away', 'xG_away', 'xGc_away', 'non_shot_xG_away', 'non_shot_xGc_away']]
away_xG["away_index"] = away_xG["away_team"] + " " + away_xG["season"].astype(str)

In [ ]:
home_xG_1 = home_xG
away_xG_1 = away_xG

In [ ]:
team_stats = home_xG.merge(away_xG, left_on='home_index', right_on='away_index', how='inner')

In [ ]:
team_stats = team_stats.drop(columns = ["home_index", "away_team", "season_y", "away_index"]).rename(columns={'GA': 'Goals'}).rename(columns={'home_team': 'team', 'season_x': 'season'})

In [ ]:
s16_17_filter = home_xG["season"] == 2016
s17_18_filter = home_xG["season"] == 2017
s18_19_filter = home_xG["season"] == 2018
s19_20_filter = home_xG["season"] == 2019
s20_21_filter = home_xG["season"] == 2020

In [ ]:
team_stats_16_17 = team_stats[s16_17_filter]
team_stats_17_18 = team_stats[s17_18_filter]
team_stats_18_19 = team_stats[s18_19_filter]
team_stats_19_20 = team_stats[s19_20_filter]
team_stats_20_21 = team_stats[s20_21_filter]

In [ ]:
avg_team_xG_home = team_stats_19_20["xG_home"].mean()
avg_team_xGc_home = team_stats_19_20["xGc_home"].mean()
avg_team_xG_away = team_stats_19_20["xG_away"].mean()
avg_team_xGc_away = team_stats_19_20["xGc_away"].mean()

In [ ]:
team_stats_19_20['home_attacking_strength'] = team_stats_19_20.apply(lambda row: row.xG_home / avg_team_xG_home, axis = 1) 
team_stats_19_20['away_attacking_strength'] = team_stats_19_20.apply(lambda row: row.xG_away / avg_team_xG_away, axis = 1) 
team_stats_19_20['home_defensive_strength'] = team_stats_19_20.apply(lambda row: row.xGc_home / avg_team_xGc_home, axis = 1) 
team_stats_19_20['away_defensive_strength'] = team_stats_19_20.apply(lambda row: row.xGc_away / avg_team_xGc_away, axis = 1) 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
team_stats_19_20

,team,season,G_home,Gc_home,xG_home,xGc_home,non_shot_xG_home,non_shot_xGc_home,G_away,Gc_away,xG_away,xGc_away,non_shot_xG_away,non_shot_xGc_away,home_attacking_strength,away_attacking_strength,home_defensive_strength,away_defensive_strength
3,AFC Bournemouth,2019,1.157895,1.578947,1.309474,1.633158,1.154737,1.666842,0.947368,1.842105,1.161579,1.767895,1.596842,1.155263,0.821327,0.849028,1.193922,1.109081
7,Arsenal,2019,1.894737,1.263158,1.512105,1.716316,1.587895,1.492632,1.052632,1.263158,1.257368,1.465263,1.446316,1.145263,0.948421,0.919044,1.254715,0.919226
9,Aston Villa,2019,1.157895,1.578947,1.354211,1.818421,1.371579,1.530526,1.000000,1.947368,1.137368,2.039474,1.877895,0.960526,0.849387,0.831332,1.329359,1.279455
13,Brighton and Hove Albion,2019,1.052632,1.421053,1.325263,1.473684,1.420000,1.247895,1.000000,1.421053,1.267368,1.677368,1.509474,1.075263,0.831230,0.926353,1.077339,1.052290
18,Burnley,2019,1.263158,1.210526,1.667895,1.409474,1.270526,1.296316,1.000000,1.421053,1.027895,1.583684,1.531053,0.834737,1.046135,0.751315,1.030397,0.993517
24,Chelsea,2019,1.578947,0.842105,2.436842,0.751053,1.817368,0.653158,2.052632,2.000000,1.706316,1.467895,1.226316,1.674211,1.528434,1.247191,0.549058,0.920877
29,Crystal Palace,2019,0.789474,1.052632,1.061579,1.247895,1.463158,1.395263,0.842105,1.578947,0.911053,1.939474,1.635789,0.890000,0.665843,0.665912,0.912275,1.216720
34,Everton,2019,1.263158,1.105263,1.444737,1.267895,1.452105,1.336842,1.052632,1.842105,1.339474,1.364211,1.120000,1.078421,0.906167,0.979056,0.926896,0.855832
45,Leicester City,2019,1.842105,0.894737,1.565789,1.291579,1.477368,1.018421,1.684211,1.263158,1.828421,1.369474,1.264737,1.523158,0.982093,1.336441,0.944210,0.859133
50,Liverpool,2019,2.736842,0.842105,2.542632,0.982632,2.192632,0.860526,1.736842,0.894737,1.921053,1.226842,0.843684,2.025263,1.594787,1.404148,0.718354,0.769654


In [ ]:
def home_team_goal_expectancy(home_team, away_team):
    home_team_filter = team_stats_19_20["team"] == home_team
    home_team_details = team_stats_19_20[home_team_filter]
    home_team_attacking_strength = home_team_details.iat[0,14]
    
    away_team_filter = team_stats_19_20["team"] == away_team
    away_team_details = team_stats_19_20[away_team_filter]
    away_team_defensive_strength = away_team_details.iat[0,17]
    
    home_team_goal_expectancy = home_team_attacking_strength * away_team_defensive_strength * avg_team_xG_home 
    return home_team_goal_expectancy

In [ ]:
def away_team_goal_expectancy(home_team, away_team):
    away_team_filter = team_stats_19_20["team"] == away_team
    away_team_details = team_stats_19_20[away_team_filter]
    away_team_attacking_strength = away_team_details.iat[0,15]
    
    home_team_filter = team_stats_19_20["team"] == home_team
    home_team_details = team_stats_19_20[home_team_filter]
    home_team_defensive_strength = home_team_details.iat[0,16]
    
    away_team_goal_expectancy = away_team_attacking_strength * home_team_defensive_strength * avg_team_xG_away
    return away_team_goal_expectancy

In [ ]:
def poisson(k, lamb):
    return (lamb**k/ scipy.special.factorial(k)) * np.exp(-lamb)

In [ ]:
def odds_calc(home_team, home_score, away_team, away_score):
    home_team_poisson = home_team_goal_expectancy(home_team, away_team)
    away_team_poisson = away_team_goal_expectancy(home_team, away_team)
    home = poisson(home_score, home_team_poisson)
    away = poisson(away_score, away_team_poisson)
    return(home * away)

In [ ]:
# for scorelines up to 8-8

scores = pd.DataFrame(data=[[0, 0],
                        [1, 1],
                        [2, 2], 
                        [3, 3],
                        [4, 4],
                        [5, 5], 
                        [6, 6],
                        [7, 7],
                        [8, 8]], columns=['home_score', 'away_score'])


outlist = [ (i, j)
    for i in scores.home_score
    for j in scores.away_score]

odds_table = pd.DataFrame(data=outlist, columns=['home_score','away_score'])

In [ ]:
# Define Home and Away teams

home_team = "Aston Villa"
away_team = "Liverpool"

In [ ]:
odds_table["odds"] = odds_calc(home_team, odds_table["home_score"], away_team, odds_table["away_score"])

#### Poisson Distribution

In [ ]:
pd.pivot_table(odds_table, values = 'odds', index='away_score', columns = 'home_score').reset_index()

# Unhash to save to csv 
# pd.pivot_table(odds_table, values = 'odds', index='away_score', columns = 'home_score').reset_index().to_csv("AVL_LIV.csv")

home_score,away_score,0,1,2,3,4,5,6,7,8
0,0,0.027432,0.028592,0.014900,0.005177,0.001349,0.000281,0.000049,7.272782e-06,9.475287e-07
1,1,0.070055,0.073017,0.038052,0.013220,0.003445,0.000718,0.000125,1.857300e-05,2.419769e-06
2,2,0.089452,0.093234,0.048588,0.016881,0.004399,0.000917,0.000159,2.371557e-05,3.089764e-06
3,3,0.076147,0.079366,0.041360,0.014370,0.003744,0.000781,0.000136,2.018803e-05,2.630181e-06
4,4,0.048615,0.050671,0.026406,0.009174,0.002391,0.000498,0.000087,1.288888e-05,1.679218e-06
5,5,0.024830,0.025880,0.013487,0.004686,0.001221,0.000255,0.000044,6.583045e-06,8.576668e-07
6,6,0.010569,0.011015,0.005740,0.001994,0.000520,0.000108,0.000019,2.801929e-06,3.650471e-07
7,7,0.003856,0.004019,0.002094,0.000728,0.000190,0.000040,0.000007,1.022211e-06,1.331780e-07
8,8,0.001231,0.001283,0.000669,0.000232,0.000061,0.000013,0.000002,3.263113e-07,4.251320e-08
